<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/%EB%B0%B0%EB%93%9C%EC%95%84%EC%9D%B4%EB%93%9C%EB%A5%BC_%EA%B5%BF%EC%95%84%EC%9D%B4%EB%94%94%EB%A1%9C_%EB%B0%94%EA%BE%B8%EA%B3%A0_10%EB%A7%8C%EA%B0%9C_%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pyspark

bg = spark.read.option("sep", "\t").csv("/test/music/profiledata_06-May-2005/artist_alias.txt").toDF('badid', 'goodid')

In [ ]:
%pyspark

bg.summary().show()

In [ ]:
%pyspark

193027 - bg.filter(bg.badid.isNull()).count()

In [ ]:
%pyspark

bg2 = bg.filter(bg.badid.isNotNull())

In [ ]:
%pyspark

bg3 = bg2.select(bg2.badid.cast("int"), bg2.goodid.cast("int"))

In [ ]:
%pyspark


## 유저_아티스트_데이터의 아티스트 아이디와 배드_굿 데이터의 배드 아이드를 연결해서 배드 아이디 찾기

result = df4.join(bg3,df4.artistid ==  bg3.badid, "left")

In [ ]:
%pyspark

result.filter(result.badid.isNotNull()).count()

In [ ]:
%pyspark

## goodid 컬럼에서 null인건 원래 artistid로 대체, null이 아닌건 그냥 쓰면 됨

## case when

## udf -> map

result2 = result.select(
    result.userid,
    when(result.goodid.isNull(), result.artistid).otherwise(result.goodid).alias('artistid'),
    result.playcount
)

In [ ]:
%pyspark

result3 = result2.groupby("userid", "artistid").agg(sum(result2.playcount).alias("playcount"))

In [ ]:
%pyspark


result4 = result3.limit(100000)

In [ ]:
%pyspark

result4.write.csv("hdfs://ai-server-01:9000/test/music/my_data.csv", header=True)
